# Simulation for theoretical MWC results

In [2]:
using Optim

"""
Mutual information (bits) as function of MWC system parameters (Martins & Swain 2011 PLoS CompBio)
* N - number of sensors
* n - number of binding sites per sensor
* K - equilibrium constant of open - close transition
* c - ratio of Kd_active / Kd_inactive
"""
function I_opt(n, N, K, c)
    return log2(sqrt((2*N)/(π*e)) * (atan(sqrt(K)) - atan(sqrt(K*(c^n)))))
end

I_opt (generic function with 1 method)

In [3]:
# Now for all settings of K and c, optimize n given N*n

K_s = logspace(0, 30, 101)
c_s = logspace(-10, 0, 101)

Ntimesn = 10000;

n_opt = zeros(length(K_s), length(c_s));
Inf_opt = zeros(length(K_s), length(c_s));
converged = zeros(length(K_s), length(c_s));

for k1 = 1:length(K_s)
    for c1 = 1:length(c_s)
        res = optimize(x->-I_opt(x, Ntimesn/x, K_s[k1], c_s[c1]), 1.0, 16)
        n_opt[k1,c1] = Optim.minimizer(res);        
        Inf_opt[k1,c1] = -Optim.minimum(res);        
        converged[k1,c1] = Optim.converged(res)
    end
end

converged;

In [4]:
using PlotlyJS

p1 = contour(x=log10(K_s), y=log10(c_s), z=round(n_opt))
p2 = contour(x=log10(K_s), y=log10(c_s), z=Inf_opt)

lo = Layout(xaxis_title="log(K)", yaxis_title="log(c)")
plot(p1, lo)

In [5]:
plot(p2, lo)

In [6]:
# Plot the binding curve given K, c, n
function Popen(alpha, n, K, c)
    return ((1.+alpha).^n)./(K.*(1.+c.*alpha).^n .+ (1.+alpha).^n)
end

# Random testing for curve plotting
inp_range = logspace(-5, 3, 1000)
outp = Popen(inp_range, 4, 10^5.44, 10^(-1.32));
inp_lin = linspace(-200, 200, 1000)*1e-3
outp_lin = Popen(21*0.048*exp(0.59*38*inp_lin), 4, 10^4.66, 10^(-1.48));

p1 = Plot(scatter(;x=inp_range, y=outp))
p2 = Plot(scatter(;x=log10(inp_range), y=log10(outp)))
p3 = Plot(scatter(;x=inp_lin*1e3, y=outp_lin))
p4 = Plot(scatter(;x=inp_lin*1e3, y=log10(outp_lin)))

plt = plot([p1; p2; p3; p4])
relayout!(plt, height=1000)
plt

# Compute BK canonical form parameters

In [7]:
# BK parameters (L0, zL, J0, zJ, Kd, C, D, E)
phi0 = [2.2e-6, 0.42, 0.1026, 0.58, 39*1e-6, 6.16,30.4,2.0];
phi1_orig = [2.67e-9, 3.87e-5, 0.0516, 0.575, 3.39e-5, 8.97, 382.5, 1.11]
phi1_log10 = [2.38e-7, 0.00016, 0.0527, 0.61, 3.794e-5, 5.41, 111.372, 2.009]

Ca=([0.0, 0.7, 4.0, 12.0, 22.0, 55.0, 70.0, 95.0]*1e-6)
V=collect(-200:25:200)*1e-3

#Constant in exp() comes from electric response (check BK functions)

# Functions to compute canonical parameter values
v_can(v, ca, phi) = phi[3].*exp(38.1728.*phi[4].*v).*phi[7].*(1.+ca./phi[5].*phi[6].*phi[8])./(1+ca./phi[5]*phi[6])
xi_can(v, ca, phi) = (1.+ca./phi[5]).^4./((1+ca./phi[5]*phi[6]).^4).*(1./(phi[1].*exp(38.1728.*phi[2].*v)))
kappa_can(ca, phi) = (1+ca./phi[5]*phi[6])./(phi[7].*(1.+ca./phi[5].*phi[6].*phi[8]))*(1+ca./phi[5]*phi[8])./(1+ca./phi[5])

# Canonical form Popen
function Popen_to_can(v, ca, phi, n)
    return ((1+v_can(v,ca,phi)).^n./((1+v_can(v,ca,phi)).^n + xi_can(v,ca,phi).*(1+kappa_can(ca,phi).*v_can(v,ca,phi)).^n))
end


# Check how much xi and v_tilde change over the voltage range
@show xi_can(V,Ca[1],phi0)[1]./xi_can(V,Ca[1],phi0)[end]
@show v_can(V,Ca[1],phi0)[end]./v_can(V,Ca[1],phi0)[1]

(xi_can(V,Ca[1],phi0))[1] ./ (xi_can(V,Ca[1],phi0))[end] = 609.7386361365911
(v_can(V,Ca[1],phi0))[end] ./ (v_can(V,Ca[1],phi0))[1] = 7016.989788502125


7016.989788502125

In [8]:
#=
# Check results of original model (works! (gives same P_opens as canonical form))

include("../BK_functions/bk_setup_script.jl"); 
Ca_grid = reshape(hcat(x_grid...)[1,:], (8,13));
V_grid = reshape(hcat(x_grid...)[2,:], (8,13));

f_model(phi; model_id=0) = reshape(BK_simulator(phi0, hcat(x_grid...),model_id=model_id), (8,13));

[Popen_to_can(V,Ca[1],phi0,4.0) f_model(phi0)'[:,1]]
=#

 # Mutual info with respect to arbitrary p($\alpha$)

In [9]:
function C(a_range, p::Function, n, N, K, c; log_func=log, num_samples=10000)
    a = linspace(a_range..., num_samples)
    
    return log_func(N) .+ log_func(K) .+
    sum(p(a).*(n.*log_func(1.+a) .+ n.*log_func(1.+c.*a) 
    .- 2.*log_func(K.*(1.+c.*a).^n .+ (1.+a).^n)))
end

C (generic function with 1 method)

In [10]:
using Calculus

# Compute linear ranges of given curves
Ca=([0.0, 0.7, 4.0, 12.0, 22.0, 55.0, 70.0, 95.0]*1e-6)
V=collect(-200:10:200)*1e-3

#= #Plotting the functions for double checking
plot([scatter(;x=V, y=log10(Popen_to_can(V,Ca[5],phi0,4.0))),
    scatter(;x=V, y=log10(Popen_to_can(V,Ca[5],phi1_log10,4.0)))]) 
=# 

# Define the function we want to set the window for (example here)
f_cur(v) = log10(Popen_to_can(v,Ca[1],phi0,4.0));

# Compute the second derivative wrt v at a bunch of locations, return appropriate range
function get_lin_range(f)
    f_sd = second_derivative(f)

    cur_deriv = []
    v1 = (-200:1:200)*1e-3;
    for v2 in v1
        cur_deriv = [cur_deriv; f_sd(v2)]
    end
    #plot([scatter(;x=v1,y=f_cur(v1)), scatter(;x=v1,y=cur_deriv./maximum(cur_deriv))])

    range = [v1[indmax(cur_deriv)], v1[indmin(cur_deriv)]]
    return range
end

# Get the voltage range where function is linear
get_lin_range(f_cur)

2-element Array{Float64,1}:
 -0.06 
  0.098

In [17]:
Ca_used = Ca[8]
phi_log10_used = phi1_log10
phi_orig_used = phi1_orig

# Corresponds to linear part of log Popen curve:
a_range_log10_v = get_lin_range(v->log10(Popen_to_can(v,Ca_used,phi_log10_used,4.0)))
 # Corresponds to linear part of Popen curve:
a_range_orig_v  = get_lin_range(v->Popen_to_can(v,Ca_used,phi_orig_used,4.0))



results_log10 = zeros(50,)
results_orig = zeros(50,)
for n1 = 1:40
    results_log10[n1] = C(v_can(a_range_log10_v, Ca_used, phi_log10_used),
        x1->(1/10000), n1, 10000/n1, 
        mean(xi_can(linspace(a_range_log10_v...,50), Ca_used, phi_log10_used)), 
        kappa_can(Ca_used, phi_log10_used))
    results_orig[n1] = C(v_can(a_range_orig_v, Ca_used, phi_orig_used),
        x1->(1/10000), n1, 10000/n1, 
        mean(xi_can(linspace(a_range_orig_v...,50), Ca_used, phi_orig_used)), 
        kappa_can(Ca_used, phi_orig_used))
end
#@show sortperm(results, rev=true)[1]
plot([Plot(scatter(;x=1:50, y=results_log10)) 
    Plot(scatter(;x=1:50, y=results_orig))])

# Above we checked information for true BK parameters
# Below we can do a sweep in canonical parameter space

In [20]:
# Properly optimizing

K_s = logspace(0, 10, 31)
c_s = logspace(-5, 0, 31)

a_range_low = v_can(a_range_log10_v, Ca_used, phi_log10_used) # Corresponds to linear part of log Popen curve
a_range_high = v_can(a_range_orig_v, Ca_used, phi_orig_used) # Corresponds to linear part of Popen curve

Ntimesn = 10000;

n_opt_low = zeros(length(K_s), length(c_s));
n_opt_high = zeros(length(K_s), length(c_s));
Inf_opt_low = zeros(length(K_s), length(c_s));
Inf_opt_high = zeros(length(K_s), length(c_s));
converged = zeros(length(K_s), length(c_s));

for k1 = 1:length(K_s)
    @show k1
    for c1 = 1:length(c_s)
        #= Proper optimization
        res_low = optimize(x->-C(a_range_low, x1->(1/10000), x, Ntimesn/x, K_s[k1], c_s[c1]), 1.0, 32)
        res_high = optimize(x->-C(a_range_high, x1->(1/10000), x, Ntimesn/x, K_s[k1], c_s[c1]), 1.0, 32)
        n_opt_low[k1,c1] = Optim.minimizer(res_low);
        n_opt_high[k1,c1] = Optim.minimizer(res_high)
        Inf_opt[k1,c1] = -Optim.minimum(res_high);        
        converged[k1,c1] = Optim.converged(res_high)
        =#
        
        # Compute C(n) for a set of ns, choose best
        results = zeros(50,)
        for n1 = 1:50
            results[n1] = C(a_range_low, x1->(1/10000), n1, Ntimesn/n1, K_s[k1], c_s[c1])
        end
        n_opt_low[k1,c1] = sortperm(results, rev=true)[1]
        Inf_opt_low[k1,c1] = sort(results, rev=true)[1]
        results = zeros(50,)
        for n1 = 1:50
            results[n1] = C(a_range_high, x1->(1/10000), n1, Ntimesn/n1, K_s[k1], c_s[c1])
        end
        n_opt_high[k1,c1] = sortperm(results, rev=true)[1]
        Inf_opt_high[k1,c1] = sort(results, rev=true)[1]
    end
end

converged;

k1 = 1
k1 = 2
k1 = 3
k1 = 4
k1 = 5
k1 = 6
k1 = 7
k1 = 8
k1 = 9
k1 = 10
k1 = 11
k1 = 12
k1 = 13
k1 = 14
k1 = 15
k1 = 16
k1 = 17
k1 = 18
k1 = 19
k1 = 20
k1 = 21
k1 = 22
k1 = 23
k1 = 24
k1 = 25
k1 = 26
k1 = 27
k1 = 28
k1 = 29
k1 = 30
k1 = 31


In [27]:
my_contours = Dict(
    :start=>0,
    :end=>8,
    :size=>1
  )

p1 = contour(x=log10(K_s), y=log10(c_s), z=n_opt_low, autocontour=false, contours=my_contours)
p2 = contour(x=log10(K_s), y=log10(c_s), z=n_opt_high, autocontour=false, contours=my_contours)

lo = Layout(xaxis_title="log(K)", yaxis_title="log(c)")
plot([Plot(p1, lo)  Plot(p2, lo)])

In [27]:
num_samples = 10000
a_range_low = [0 1] # Corresponds to BK log10(Popen) < -3.5, ~ 80mV
a_range_med = [0 10] # Corresponds to BK log10(Popen) < -1, ~ 140mV
a_range_full = [1 95]
n1 = 21; n2=4; K = 10^4.66; c= 10^-1.33
p1 = scatter(;x=linspace(a_range_low..., num_samples), y=Popen(linspace(a_range_low..., num_samples), n1, K, c))
p2 = scatter(;x=linspace(a_range_full..., num_samples), y=Popen(linspace(a_range_full..., num_samples), n2, K, c))

plot([Plot(p1); Plot(p2)])

In [50]:
# Voltage - Popen plot
plot(scatter(;x=inp_lin*1000, y=outp_lin))

In [62]:
# Voltage - log10(Popen) plot
plot(scatter(;x=inp_lin*1000, y=log10(outp_lin)))

In [48]:
# What "concentration" (in standardized MWC) corresponds to the the given voltage
v_given = 200*1e-3
21*0.048*exp(0.59*38*v_given)

89.29702473133403

In [31]:
using Physical

In [33]:
0.59 * ElectronVolt./(k_boltzmann*304*Kelvin)

38.17276086450309

# Fit 